In [1]:
import nltk
from nltk.classify import ClassifierI
from statistics import mode

In [2]:
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

In [3]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [169]:
from nltk.classify import ClassifierI
from statistics import mode

In [170]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [171]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [172]:
movie_reviews.categories()

['neg', 'pos']

In [173]:
movie_reviews.fileids()[1]
#movie_reviews.words().fileids()[1]

'neg/cv001_19502.txt'

In [138]:
documents = [(list(movie_reviews.words(fileid)),category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

In [174]:
movie_reviews.words('neg/cv001_19502.txt')

['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...]

In [175]:
random.shuffle(documents)

In [176]:
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())

In [177]:
all_words = nltk.FreqDist(all_words)

In [178]:
word_features = list(all_words.keys())[:3000]

In [179]:
all_words

FreqDist({',': 77717, 'the': 76529, '.': 65876, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, "'": 30585, 'is': 25195, 'in': 21822, ...})

In [180]:
print(all_words.most_common(15))

[(',', 77717), ('the', 76529), ('.', 65876), ('a', 38106), ('and', 35576), ('of', 34123), ('to', 31937), ("'", 30585), ('is', 25195), ('in', 21822), ('s', 18513), ('"', 17612), ('it', 16107), ('that', 15924), ('-', 15595)]


In [181]:
#remoce stop words
from nltk.corpus import stopwords

In [182]:
stop_words = set(stopwords.words('english'))

In [183]:
filter_all_words = [w for w in all_words if not w  in stop_words]

In [184]:
filter_all_words = nltk.FreqDist(filter_all_words)

In [185]:
word_features_filter = list(filter_all_words.keys())[:3000]

In [186]:
print(filter_all_words.most_common(15))

[('plot', 1), (':', 1), ('two', 1), ('teen', 1), ('couples', 1), ('go', 1), ('church', 1), ('party', 1), (',', 1), ('drink', 1), ('drive', 1), ('.', 1), ('get', 1), ('accident', 1), ('one', 1)]


In [187]:
filter_all_words

FreqDist({'plot': 1, ':': 1, 'two': 1, 'teen': 1, 'couples': 1, 'go': 1, 'church': 1, 'party': 1, ',': 1, 'drink': 1, ...})

In [188]:
len(all_words)

39768

In [189]:
len(filter_all_words)

39617

In [190]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features_filter:
        features[w] = (w in words)
    return features
            

In [156]:
#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))


In [191]:
featuresets = [(find_features(rev),category) for (rev,category) in documents]

In [213]:
training_set = featuresets[:1900]
testing_set = featuresets[1900:]
len(testing_set)

100

In [193]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)


MNB_classifier accuracy percent: 83.0


In [194]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)


BernoulliNB_classifier accuracy percent: 83.0


In [195]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)


LogisticRegression_classifier accuracy percent: 84.0


In [196]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)


C:\Users\asinha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 81.0


In [197]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


LinearSVC_classifier accuracy percent: 82.0
NuSVC_classifier accuracy percent: 85.0


In [200]:
voted_classifier = VoteClassifier(NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

    

In [201]:

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

voted_classifier accuracy percent: 83.0


In [202]:
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

Classification: pos Confidence %: 80.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0


In [210]:
len(testing_set)

100

In [211]:
for i in range(0,100):
    print("Classification:", voted_classifier.classify(testing_set[i][0]), "Confidence %:",voted_classifier.confidence(testing_set[i][0])*100)

Classification: pos Confidence %: 80.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 80.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 80.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 80.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Clas